In [ ]:
import cv2
import numpy as np
import struct
import time
import matplotlib.pyplot as plt
import os
import re
import sys
from pathlib import Path

from read_cine import read_cine, convert_cine_to_avi
from track_object import track_object, detect_chamber, update_tracking_result, show_tracking_results

In [6]:
cine_path = r"F:\AUG2025\P24\00_He1kG430G_5800A_K-25_shot007.cine"
avi_path = cine_path.replace('.cine', '.avi')

tarr, frarr, dt = read_cine(cine_path)

convert_cine_to_avi(frarr, avi_path)

Reading .cine file...
starting frame number: 0
total frames: 2001
frame rate: 20000
Done reading .cine file (10.0 s)
Converting to F:\AUG2025\P24\00_He1kG430G_5800A_K-25_shot007.avi...
Progress: 100/2001 frames
Progress: 200/2001 frames
Progress: 300/2001 frames
Progress: 400/2001 frames
Progress: 500/2001 frames
Progress: 600/2001 frames
Progress: 700/2001 frames
Progress: 800/2001 frames
Progress: 900/2001 frames
Progress: 1000/2001 frames
Progress: 1100/2001 frames
Progress: 1200/2001 frames
Progress: 1300/2001 frames
Progress: 1400/2001 frames
Progress: 1500/2001 frames
Progress: 1600/2001 frames
Progress: 1700/2001 frames
Progress: 1800/2001 frames
Progress: 1900/2001 frames
Progress: 2000/2001 frames
Progress: 2001/2001 frames
Conversion complete. Saved to F:\AUG2025\P24\00_He1kG430G_5800A_K-25_shot007.avi


In [ ]:
parr, frarr, cf = track_object(avi_path)

In [ ]:
frame_num = 1861

cap = cv2.VideoCapture(avi_path)
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
ret, frame = cap.read()
if not ret:
    raise ValueError(f"Could not read frame")

# Detect chamber
(cx, cy), chamber_radius = detect_chamber(frame)

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Display the original frame
plt.figure(figsize=(15, 5))
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

# Draw the chamber as a circle
chamber_circle = plt.Circle((cx, cy), chamber_radius, fill=False, color='green', linewidth=2)
plt.gca().add_patch(chamber_circle)

plt.title(f"t={tarr[frame_num] * 1e3:.3f}ms")
plt.axis('off')

cap.release()

print(tarr[frame_num])

In [ ]:
v = 9.8*np.sqrt(2*1/9.8) # velocity at center of chamber in m/s

t1 = 0.027 - tarr[cf]
t2 = 0.037 - tarr[cf]
print(t1)
print(t2)

x1 = v*t1 + 0.5*9.8*t1**2
x2 = v*t2 + 0.5*9.8*t2**2

print(x1)
print(x2)

In [ ]:
tr_ifn = r"E:\good_data\kapton\He3kA_B380G800G_pl0t20_uw15t35\tracking_results.npy"
show_tracking_results(tr_ifn)

In [ ]:
update_tracking_result(tr_ifn, cine_path, cf_new=frame_num, ct_new=tarr[frame_num])